## Intro PandaSQL

In [2]:
import pandas as pd
from pandasql import sqldf
from pandasql import load_meat, load_births          # Importing Data
# Bring data in Python environment as pandas DataFrame
meat = load_meat()
births = load_births()

In [3]:
meat.head()

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01,751.0,85.0,1280.0,89.0,NaN,NaN,NaN
1,1944-02-01,713.0,77.0,1169.0,72.0,NaN,NaN,NaN
2,1944-03-01,741.0,90.0,1128.0,75.0,NaN,NaN,NaN
3,1944-04-01,650.0,89.0,978.0,66.0,NaN,NaN,NaN
4,1944-05-01,681.0,106.0,1029.0,78.0,NaN,NaN,NaN


In [4]:
# specify globals() or locals() using the following helper function
mysql = lambda q: sqldf(q, globals())
mysql("SELECT * FROM meat LIMIT 5;")

,date,beef,veal,pork,lamb_and_mutton,broilers,other_chicken,turkey
0,1944-01-01 00:00:00.000000,751.0,85.0,1280.0,89.0,None,None,None
1,1944-02-01 00:00:00.000000,713.0,77.0,1169.0,72.0,None,None,None
2,1944-03-01 00:00:00.000000,741.0,90.0,1128.0,75.0,None,None,None
3,1944-04-01 00:00:00.000000,650.0,89.0,978.0,66.0,None,None,None
4,1944-05-01 00:00:00.000000,681.0,106.0,1029.0,78.0,None,None,None


# Join

In [5]:
query = '''
SELECT m.date, m.beef, m.veal, m.pork, b.births
FROM meat AS m
INNER JOIN
births AS b
ON m.date = b.date;
'''

mysql(query)

,date,beef,veal,pork,births
0,1975-01-01 00:00:00.000000,2106.0,59.0,1114.0,265775
1,1975-02-01 00:00:00.000000,1845.0,50.0,954.0,241045
2,1975-03-01 00:00:00.000000,1891.0,57.0,976.0,268849
3,1975-04-01 00:00:00.000000,1895.0,60.0,1100.0,247455
4,1975-05-01 00:00:00.000000,1849.0,59.0,934.0,254545
...,...,...,...,...,...
402,2012-07-01 00:00:00.000000,2200.8,9.5,1721.8,368450
403,2012-08-01 00:00:00.000000,2367.5,10.1,1997.9,359554
404,2012-09-01 00:00:00.000000,2016.0,8.8,1911.0,361922
405,2012-10-01 00:00:00.000000,2343.7,10.3,2210.4,347625


In [6]:
query = '''SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
              LIMIT 5;
    '''

mysql(query)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0


In [8]:
query = '''SELECT
            strftime('%Y', date) as year
           , SUM(beef) as beef_total
           FROM
              meat
           GROUP BY
              year
    '''

mysql(query).head(10)

,year,beef_total
0,1944,8801.0
1,1945,9936.0
2,1946,9010.0
3,1947,10096.0
4,1948,8766.0
5,1949,9142.0
6,1950,9248.0
7,1951,8549.0
8,1952,9337.0
9,1953,12055.0


In [10]:
#executing union all statements
query = """
        SELECT
            date
            , 'beef' AS meat_type
            , beef AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'veal' AS meat_type
            , veal AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'pork' AS meat_type
            , pork AS value
        FROM meat
        UNION ALL
        SELECT
            date
            , 'lamb_and_mutton' AS meat_type
            , lamb_and_mutton AS value
        FROM meat
        ORDER BY 1
    """
mysql(query).head(10)

,date,meat_type,value
0,1944-01-01 00:00:00.000000,beef,751.0
1,1944-01-01 00:00:00.000000,veal,85.0
2,1944-01-01 00:00:00.000000,pork,1280.0
3,1944-01-01 00:00:00.000000,lamb_and_mutton,89.0
4,1944-02-01 00:00:00.000000,beef,713.0
5,1944-02-01 00:00:00.000000,veal,77.0
6,1944-02-01 00:00:00.000000,pork,1169.0
7,1944-02-01 00:00:00.000000,lamb_and_mutton,72.0
8,1944-03-01 00:00:00.000000,beef,741.0
9,1944-03-01 00:00:00.000000,veal,90.0


# Nested Query

In [12]:
# use queries within queries
query = """
    SELECT
        m1.date
        , m1.beef
    FROM
        meat m1
    WHERE m1.date IN
        (SELECT
            date
        FROM meat
        WHERE
            beef >= broilers
        ORDER BY date)
"""
mysql(query)

,date,beef
0,1960-01-01 00:00:00.000000,1196.0
1,1960-02-01 00:00:00.000000,1089.0
2,1960-03-01 00:00:00.000000,1201.0
3,1960-04-01 00:00:00.000000,1066.0
4,1960-05-01 00:00:00.000000,1202.0
...,...,...
416,1995-08-01 00:00:00.000000,2316.0
417,1995-09-01 00:00:00.000000,2220.0
418,1995-11-01 00:00:00.000000,2098.0
419,1996-05-01 00:00:00.000000,2302.0
